# Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train three machine learning classifiers: CART, Support Vector Machine and Random Forest. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>
**NOTE:** The input image needs to have the bands B1, B2, B3, B4, B5, B8, B11, B12 to apply masks correctly. The classifications will use only the bands B1, B2, B3, B4 and Blue/Green, which represent the bands that penetrates the most into the water column.

Script by: Luis Lizcano-Sandoval<br/>
College of Marine Sciences, University of South Florida<br/>
Updated: 11/16/2020

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds, land, and deep areas >20m
3. Apply Depth-Invariant Index (generates band-ratios B1B2, B1B3, B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (CART, SVM and RF)
6. Get confusion matrices and accuracies
7. Export output to EE Assets (.tiff)
8. Save matrices in local computer (.xlxs)

Load required libraries:

In [ ]:
import ee
import pandas as pd
import xlsxwriter
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
import datetime
from functions import CloudScore6S,landMaskFunction,DII
from IPython.display import display, Image

ee.Initialize()
print('EE API version: ',ee.__version__)

Initial Settings and Metadata:

In [ ]:
## Define the source of your image, if from your EE Assets or EE Collections:
# imageSource = 'assets'  ## EE Assets
imageSource = 'ee'      ## EE Collections

## Define the type of satellite
# satellite = 'Sentinel2'
satellite = 'Landsat8'
# satellite = 'Landsat7'
# satellite = 'Landsat5'

## Some settings:
#imageID = 'LT05_016041_19890205'
imageID = 'LC08_017041_20151103'
# imageID = '20190208T160421_20190208T161051_T17RLL' ## Image ID
boaFolder = 'FL_19' ## Name of folder with BOA images in assets.
exportFolder = 'FL_seasonality' ## Name of EE folder to save the final output
dataFolder = 'Ground-points-19' ## EE Folder with Ground-truth points
smoothStr = '_raw_' # Smooth classified pixels or not? options: '_smooth_' or '_raw_'

<font size="4">**1. Import files:**</font>

Import BOA images and extract metadata:

In [ ]:
## If the image source is your asset, then define the folder where the satellite image is:
if 'assets'== imageSource:
    ## Load BOA image from assets:
    if 'Sentinel' in satellite:
        imageTarget = ee.Image("users/lizcanosandoval/BOA/Sentinel/"+boaFolder+'/'+imageID)
    elif 'Landsat' in satellite:
        imageTarget = ee.Image("users/lizcanosandoval/BOA/Landsat/"+boaFolder+'/'+imageID)

## If the image source is an EE collection, then define the satellite collection:
if 'ee'== imageSource:
    ## Load BOA image collection from assets:
    if 'Sentinel' in satellite:
        image = ee.Image("COPERNICUS/S2_SR/"+imageID)
    elif 'Landsat8' == satellite:
        image = ee.Image("LANDSAT/LC08/C01/T1_SR/"+imageID)
    elif 'Landsat7' == satellite:
        image = ee.Image("LANDSAT/LE07/C01/T1_SR/"+imageID)
    elif 'Landsat5' == satellite:
        image = ee.Image("LANDSAT/LT05/C01/T1_SR/"+imageID)

## Get image metadata:
if 'assets'== imageSource:
    imageSat = imageTarget.get('satellite').getInfo() #Image satellite
    imageTile = imageTarget.get('tile_id').getInfo() #Image tile id
    imageDate = imageTarget.get('date').getInfo() #Image date
    imageGeometry = imageTarget.geometry() #Tile geometry.

if 'ee'== imageSource:
    if 'Sentinel' in satellite:
        imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
        imageSat = imageTarget.get('SPACECRAFT_NAME').getInfo() #Image satellite
        imageTile = imageTarget.get('MGRS_TILE').getInfo() #Image tile id
        ee_date = imageTarget.get('GENERATION_TIME').getInfo()
        imageDate = str(datetime.datetime.utcfromtimestamp(ee_date/1000.0)) #Image date
        imageGeometry = imageTarget.geometry() #Tile geometry.
    elif 'Landsat8' == satellite:
        imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
        #imageSat = imageTarget.get('SATELLITE').getInfo() #Image satellite
        imageSat = satellite
        imageTile = str(imageTarget.get('WRS_PATH').getInfo())+str(imageTarget.get('WRS_ROW').getInfo()) #Image tile id
        imageDate = imageTarget.get('SENSING_TIME').getInfo()
        imageGeometry = imageTarget.geometry() #Tile geometry.
        
if 'Sentinel' in imageSat:
    imageScale = 10 # Sentinel resolution
else:
    imageScale = 30 # Landsat resolution


print('Satellite: ',imageSat)
print('Tile: ',imageTile)
print('Date: ',imageDate)
print(imageDate[0:4])

Visualize image

In [ ]:
## Define RGB bands:
if 'Sentinel' or 'Landsat8' in imageSat:
    rgb = ['B4','B3','B2']
else:
    rgb = ['B3','B2','B1'] ##Landsat7/5

## Setting for displaying images:
def displaySettings(image, channels):
    img = Image(url=image.select(channels).getThumbUrl({
        'dimensions': '500x500',
        'min':0,
        'max':0.2,
        'gamma':1.8
        }))
    return display(img)

## Display image:
originalImage = displaySettings(imageTarget, rgb)

Import Ground-Points [Training and Validation] and sandy areas [polygons] for water column correction [DIV]:

In [ ]:
# Sandy areas
sand_areas = ee.FeatureCollection("users/lizcanosandoval/ground-points/Sand")

# Ground-Points
groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/"+dataFolder)

Import other collections:

In [ ]:
## ETOPO1: Global 1 Arc-Minute Elevation:
# etopo = ee.Image("NOAA/NGDC/ETOPO1")
# etopo = etopo.select('bedrock') ## Select the bathymetry band

## Florida Bathymetry [NOAA-90m res]
# bathymetry = ee.Image("users/lizcanosandoval/Bathymetry_FL")

## Florida Land [from GADM-HiRes]
#gadm_FL = ee.FeatureCollection("users/lizcanosandoval/gadm36_FL")
gadm_FL = ee.FeatureCollection("users/lizcanosandoval/Florida_10m")##Created from NDWI using Sentinel-2 imagery

## FAO Global Administrative Unit Layers 2015, Country Boundaries
# FAO = ee.FeatureCollection("FAO/GAUL/2015/level0")

## Filter country. To select country by GAUL code see: http://www.fao.org/countryprofiles/iso3list/en/
# country = FAO.filter(ee.Filter.eq('ADM0_CODE',259))

## Florida's Seagrass Habitats (FWC database)
#seagrass = ee.FeatureCollection("users/lizcanosandoval/Seagrass_Habitat_Florida"),

## Sentinel-2 Tiles over seagrass areas:
#tiles = ee.FeatureCollection("users/lizcanosandoval/S2_tiles_surf-to-shallow_line")

<font size="4">**Prepare and mask bathymetry data:**</font>

In [ ]:
## Mask depth ranges from the Etopo collection [from -25 to 1 meter]
# etopo_masked = ee.Image(etopo).updateMask(etopo.lt(1).And(etopo.gt(-25)))

# ## Resample ETOPO
# etopo_resample = etopo_masked.reproject(**{\
#                                          'crs': 'EPSG:32617',
#                                          'scale': 90,
#                                         })
    
# ## Define a boxcar or low-pass kernel.
# kernel = ee.Kernel.square(**{\
#                            'radius': 3, 
#                            'units': 'pixels', 
#                            'normalize': True,
#                           })

# ## Smooth raster
# #etopo_kernel = etopo_resample.resample('bilinear')
# etopo_kernel = etopo_resample.convolve(kernel)

# ## Mask the FL bathymetry collection (NOAA):
# ## Mask depth ranges from the FL bathymetry collection [Mask depth range from -20 to 2 meters]
# bathy_masked = ee.Image(bathymetry).updateMask(bathymetry.lt(2).And(bathymetry.gt(-25)))

Clip bathymetry to the tile geometry/bounds:

In [ ]:
## Clip bathymetry layers to tile geometry
# bathyBand = bathy_masked.clip(imageGeometry) ##Florida bathymetry clip (90m res)
# etopo_clip = etopo_kernel.clip(imageGeometry) ##Etopo clip + kernel (90m resampled)

As the bathymetry dataset from NOAA has small gaps in some areas we need to fill them using the resampled ETOPO collection:

In [ ]:
## Vectorize (to FeatureCollection of polygons) the bathymetry collection,
## which is a raster image.
# bathyVector = bathyBand.toByte().reduceToVectors(**{
#   'reducer': ee.Reducer.countEvery(),
#   'crs': 'EPSG:4326',
#   'geometry': None,
#   'eightConnected': False,
#   'labelProperty': 'bathymetry',
#   'scale': 1000,
#   'geometryType': 'polygon',
#   'maxPixels': 1e9
# })

# ## Extract the bathymetry values from the Etopo layer corresponding to the gap 
# ## found in the FL Bathymetry Data.
# bathyOutside = ee.Image.constant(1).clip(bathyVector).mask().Not()
# bathyGap = etopo_clip.updateMask(bathyOutside)
# bathyVectorGap = bathyGap.toByte().reduceToVectors(**{
#   'reducer': ee.Reducer.countEvery(),
#   'crs': 'EPSG:4326',
#   'geometry': None,
#   'eightConnected': False,
#   'labelProperty': 'bathymetry',
#   'scale': 1000,
#   'geometryType': 'polygon',
#   'maxPixels': 1e9
# });
# bathyVectorGap = bathyVectorGap.geometry().buffer(50) ##Fill gap and extend a buffer of 50m
# bathyOutside2 = ee.Image.constant(1).clip(bathyVectorGap).mask()
# bathyGap2 = etopo_clip.updateMask(bathyOutside2) ##Clip the gap in the ETOPO collection.

# #Finally, fill the gap in the NOAA bathymetry dataset. This is the new NOAA's bathymetry raster layer:
# bathyFilled = ee.ImageCollection([bathyBand,bathyGap2.select(['bedrock'],['b1'])]).mosaic()

<font size="4">**2. Apply masks to image:**</font>

Cloud mask:

In [ ]:
## Recommended Threshold values for
## *Sentinel: 2
## *Landsat: 5
if 'Sentinel' in imageSat:
    threshold = 5
else:
    threshold = 5

## Apply cloud mask
cloudMask = CloudScore6S(imageSat, imageTarget, threshold)

## Display image:
#cloudImage = displaySettings(cloudMask, rgb)
#Map.addLayer(cloudMask, vizParams, 'cloudMask')
#Map

Land mask:

In [ ]:
## Apply land mask
landMask = landMaskFunction(cloudMask, gadm_FL)
#landMask = cloudMask.updateMask(gadm_FL.mosaic()) ##Use if gadm is image collection

## Display image:
#landImage = displaySettings(landMask, rgb)
#Map.addLayer(landMask, vizParams, 'landMask')
#Map

Bathymetry mask:

In [ ]:
## Apply bathymetry mask
bathyMask = landMask##.clip(bathyVector) ##Using the NOAA dataset: bathyVector

## Display image:
#bathyImage = displaySettings(bathyMask, rgb)

Turbidity mask:

In [ ]:
## Turbidity masking is based on the red band reflectances. Based on own observations (in South Florida), values higher than 0.02-0.03 indicates
## turbid waters, but sometimes may indicate shallow seagrass banks. So the below algorithm try to separate shallow seagrass
## from turbidity.

## Set parameter values
if 'Sentinel' in imageSat:
    red_band = 'B4' #Red seems work better in this case than B5.
    red_thr_inf = 0.025
    red_thr_sup = 0.2
    green_band = 'B3'
    green_thr = 0.15
    blue_band = 'B2'
    blue_thr = 0.11
elif 'Landsat8' in imageSat:
    red_band = 'B4'
    red_thr_inf = 0.025
    red_thr_sup = 0.2
    green_band = 'B3'
    green_thr = 0.15
    blue_band = 'B2'
    blue_thr = 0.11
else:
    red_band = 'B3'
    red_thr_inf = 0.03 #Landsat5/7 are less sensitive
    red_thr_sup = 0.2
    green_band = 'B2'
    green_thr = 0.15
    blue_band = 'B1'
    blue_thr = 0.11

## Select the red band
selectRedBand = bathyMask.select('B4')

## Identify turbid areas first (pixel values higher than the threshold)
turbidMask = selectRedBand.gt(red_thr_inf)

## Apply thresholds on red, green and blue bands
maskRed = selectRedBand.gt(red_thr_inf).And(selectRedBand.lt(red_thr_sup))
imageMaskRed = bathyMask.mask(maskRed)
maskGreen = imageMaskRed.select(green_band).lt(green_thr)
imageMaskGreen = imageMaskRed.mask(maskGreen)
maskBlue = imageMaskGreen.select(blue_band).lt(blue_thr) ##Shallow seagrass

## Final mask (excluding seagrass/including turbid water)
turbidImage = bathyMask.mask(turbidMask) ## Turbidity
seagrassImage = bathyMask.mask(maskBlue).mask().Not() ## Shallow seagrass (inverse mask)
excludeSeagrass = turbidImage.updateMask(seagrassImage) ## Turbidity minus shallow seagrass
finalMaskImage = bathyMask.mask(excludeSeagrass)
finalMask = finalMaskImage.mask().Not()

## Final Image
finalImage = bathyMask.updateMask(finalMask)

## Display image:
#cleanImage = displaySettings(finalImage, rgb)
#Map.addLayer(finalImage, vizParams, 'finalImage')
#Map

<font size="4">**3. Water column correction:**</font>

In [ ]:
## Filter sand polygons by tile/area:
sand = ee.FeatureCollection(sand_areas).filterBounds(imageGeometry)
print('Number of sand polygons: ',sand.size().getInfo())

## Run the Depth-Invariant Index Function
imageDII = DII(finalImage, imageScale, sand)
#print(imageDII.getInfo())

## Display one of the bands ['B1B2', 'B1B3', 'B2B3']
imgDII = Image(url=imageDII.select('B2B3').getThumbUrl({
    'dimensions': '500x500',
    'min':-3,
    'max':1,
    'gamma':1.5
    }))
#display(imgDII)

<font size="4">**4. Sampling Data:**</font>

Classes are:
* 0: Softbottom
* 1: Hardbottom
* 2: Seagrass
* 3: Sparse seagrass //if available

In [ ]:
## Filter ground points by tile geometry and display classes
filterPoints = ee.FeatureCollection(groundPoints).filterBounds(imageGeometry)
print('Classes: ', filterPoints.aggregate_array('class').distinct().getInfo())
print('Ground Points per Class:', filterPoints.aggregate_histogram('class').getInfo())

totalPoints = filterPoints.size()
print('Total points:', totalPoints.getInfo())

Select the bands to sample:

In [ ]:
## Select bands to sample. The B/G band is B2B3 in Sentinel-2 and Landsat-8, and B1B2 for Landsat-7/5
if 'Sentinel' in imageSat or 'Landsat8' in imageSat:
    bandsClass = ['B1','B2', 'B3', 'B4','B2B3']
    bg = ['B2B3']
else:
    bandsClass = ['B1','B2', 'B3', 'B1B2']
    bg = ['B1B2']
    
## Add bands of interest to sample training points:
imageClassify = finalImage.addBands(imageDIV.select(bg)).select(bandsClass)
print('Bands to sample:',imageClassify.bandNames().getInfo())
##Other configurations:
#imageClassify = imageDIV.select(bandsClass); // Depth invariant Bands B1B2,B1B3,B2B3.
#imageClassify = imageKD.select(bandsClass)//.addBands(bathyFilled); //KD corr B1-B4, with or without bathymetry.



**MAKE TRAINING WITHIN THE 5KM BOUNDS OF SEAGRASS HABITATS AND OPTICALLY SHALLOW AREAS**

In [ ]:
seagrass_buffer = ee.FeatureCollection("users/lizcanosandoval/Seagrass_Habitat_Florida_buff5k")
#shallowMask = ee.ImageCollection("users/lizcanosandoval/Shallow_Mask_FL")
#specificTurbidMask = ee.ImageCollection("users/lizcanosandoval/TurbidityMask")
#listShallowMask = shallowMask.aggregate_array('system:index').getInfo()
#listTurbidMask = specificTurbidMask.aggregate_array('system:index').getInfo()

imageClassify = imageClassify.clip(seagrass_buffer) #For polygons

# ##If image has a specific shallow water mask then apply it:
# if imageTile in listShallowMask:
#     imageClassify = imageClassify.updateMask(shallowMask.mosaic())

# ##If image has a specific turbidity mask then apply it:
# if imageID in listTurbidMask:
#     filterTurbidMask = ee.Image(specificTurbidMask.filter(ee.Filter.eq('system:index',imageID)).first())
#     mask = filterTurbidMask.Not()
#     imageClassify = imageClassify.updateMask(mask)

#buffer = ee.FeatureCollection("users/lizcanosandoval/Turkey_coastline_buffer_1km")
# clipper = ee.FeatureCollection("users/lizcanosandoval/ground-points/AOI_SpringsCoast")
# imageClassify = imageClassify.clip(clipper)

##imageGeometry = imageClassify.geometry()
#cleanImage = displaySettings(imageClassify, rgb)
#Map.addLayer(imageClassify, vizParams, 'imageClassify')
#Map

**APPLY SMOOTHER IF SET**

In [ ]:
## Define a boxcar or low-pass kernel (Used if want to smooth the image)
smooth = ee.Kernel.euclidean(**{
    'radius': 1, 
    'units': 'pixels', 
    'normalize': True
})

## Apply smoother if set:
if 'smooth' in smoothStr:
    imageClassify = imageClassify.convolve(smooth)

Sample training and validation data:

In [ ]:
## Sample multi-spectral data using all ground points.
samplingData = imageClassify.sampleRegions(**{
    'collection': filterPoints,
    'properties': ['class'],
    'scale': imageScale})

## Add random numbers to each feature (from 0 to 1).
randomData = samplingData.randomColumn("random",0)

## Split ground data in training (~70%) and validation (~30%) points
trainingData = randomData.filter(ee.Filter.lt("random",0.7))
validationData = randomData.filter(ee.Filter.gte("random", 0.7))

print('Training Points per Class:', trainingData.aggregate_histogram('class').getInfo())
print('Validation Points per Class:', validationData.aggregate_histogram('class').getInfo())
print('Training Samples (70%):',trainingData.size().getInfo())
print('Validation Samples (30%):',validationData.size().getInfo())

<font size="4">**5. Train models and Classify:**</font>

In [ ]:
## Train SVM classifier
SVM = ee.Classifier.libsvm(**{
   'kernelType': 'RBF',
   'gamma': 100,
   'cost': 100
})
trainSVM = SVM.train(**{
   'features': trainingData,
   'classProperty': 'class',
   'inputProperties': bandsClass
})

#### Classify the image using the trained classifier
classifiedSVM = imageClassify.classify(trainSVM)

Clean and Display classified images:

In [ ]:
## Load seagrass mask 2019 and clip
seagrass_mask = ee.ImageCollection("users/lizcanosandoval/SeagrassMask_FL").mosaic()
classifiedSVM = classifiedSVM.updateMask(seagrass_mask) #For raster

## Define a palette for the distinct classes
classPalette = ['#3090C7','#CD7F32','#004E00']#,'#78F878']

imgSVM = Image(url=classifiedSVM.getThumbUrl({
    'dimensions': '500x500',
    'min':0,
    'max':2,
    'palette': classPalette
    }))

display(imgSVM)

<font size="4">**6. Get accuracy matrices:**</font>

Training accuracies:

In [ ]:
## Get a confusion matrix representing resubstitution accuracy.
## {Resubstitution error is the error of a model on the training data.}
## Axis 0 (first level) of the matrix correspond to the input classes (columns), 
## and axis 1 (second level) to the output classes (rows).
matrixTrainingSVM = trainSVM.confusionMatrix()

#print('SVM Training Confusion Matrix: ', matrixTrainingSVM.getInfo())

print('SVM Training overall accuracy: ', round((matrixTrainingSVM.accuracy().getInfo()), 4))

Validation accuracies:

In [ ]:
## Calculate accuracy using validation data
## Classify the image using the trained classifier
validationSVM = validationData.classify(trainSVM)

## Get a confusion matrix representing expected accuracy (Using validation points - 30%), where:
#  0: Softbottom
#  1: Hardbottom
#  2: Dense Seagrass
#  3: Spare Seagrass

## Axis 0 (the rows) of the matrix correspond to the actual values, 
## and Axis 1 (the columns) to the predicted values.
errorMx = {'actual': 'class', 'predicted': 'classification'}
errorMatrixSVM = validationSVM.errorMatrix(**errorMx)

#print('SVM Validation Error Matrix: ', errorMatrixSVM.getInfo())

print('SVM validation overall accuracy: ', round((errorMatrixSVM.accuracy().getInfo()), 4))

User and Producer accuracies:

In [ ]:
## Estimate user and producer accuracies
producerAccuracySVM = errorMatrixSVM.producersAccuracy()

userAccuracySVM = errorMatrixSVM.consumersAccuracy()

#print('Producer Accuracy SVM: ',producerAccuracySVM.getInfo())
#print('User Accuracy SVM: ',userAccuracySVM.getInfo())

Print accuracies as Pandas format:

In [ ]:
## Create a pandas dataframe with producer and user accuracies:
rowIndex = {0:'Sb', 1:'Hb', 2:'Dn', 3:'Sp'}
dfPA_SVM = pd.DataFrame(producerAccuracySVM.getInfo(), columns=['Producer'])
dfUA_SVM = pd.DataFrame(userAccuracySVM.getInfo()).transpose()

PU_SVM = pd.concat([dfPA_SVM, dfUA_SVM.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)

print('\n SVM: \n', PU_SVM)

Kappa coefficients:

In [ ]:
# The Kappa Coefficient is generated from a statistical test to evaluate the accuracy 
# of a classification. Kappa essentially evaluate how well the classification performed 
# as compared to just randomly assigning values, i.e. did the classification do better 
# than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that 
# the classification is no better than a random classification. A negative number 
# indicates the classification is significantly worse than random. A value close to 1 
# indicates that the classification is significantly better than random.

kappaSVM = errorMatrixSVM.kappa()

print('Kappa SVM: ', round((kappaSVM.getInfo()), 4))

<font size="4">**7. Export Classified Images:**</font>

In [ ]:
# Set the scale properly
scale = []
sat = []
method = ['SVM']
classifiedCollection = ee.ImageCollection([classifiedSVM])
classifiedList = classifiedCollection.toList(classifiedCollection.size())
classifiedSize = classifiedList.size().getInfo()
print('Wait for submission')

for i in range(classifiedSize):
    
    # Rename satellite
    if 'Sentinel' in imageSat:
        sat = 'Sentinel'
    else:
        sat = 'Landsat'
        
    ## Select image
    image = ee.Image(classifiedList.get(i))
    
    # set some properties for export
    output = image.set({'satellite': imageSat,
                   'tile_id': imageTile,
                   'file_id': imageID,                                               
                   'date': imageDate,
                   'year': imageDate[0:4],
                   'classifier': method[i],
                   'generator': 'Lizcano-Sandoval',
                        })

    # define YOUR assetID. (This do not create folders, you need to create them manually)
    assetID = 'users/lizcanosandoval/Seagrass/'+sat+'/'+exportFolder+'/' ##This goes to an ImageCollection folder
    fileName = imageID+smoothStr+ method[i]
    path = assetID + fileName
    
    ## Batch Export to Assets
    ee.batch.Export.image.toAsset(\
        image = ee.Image(output),                                                    
        description = method[i] +smoothStr+ imageID,
        assetId = path,
        region = imageGeometry.buffer(10),                                      
        maxPixels = 1e13,
        scale = imageScale).start()
    print('Classified Image '+str(i+1)+': '+imageID +smoothStr+ method[i]+' submitted...')
print('All images submitted!')

<font size="4">**8. Save Matrices to Working Directory:**</font>

Extract values from each matrix

In [ ]:
SVM_trainingMatrix = matrixTrainingSVM.array().getInfo()
SVM_trainingAccuracy = matrixTrainingSVM.accuracy().getInfo()
SVM_errorMatrix = errorMatrixSVM.array().getInfo()
SVM_errorAccuracy = errorMatrixSVM.accuracy().getInfo()
SVM_producerAccuracy = producerAccuracySVM.getInfo()
SVM_userAccuracy = userAccuracySVM.getInfo()
SVM_kappa = kappaSVM.getInfo()

print('These matrices needs to be reformatted, e.g.: ', SVM_trainingMatrix)

Convert matrices to pandas dataframes

In [ ]:
#Training Matrices
TM_SVM = pd.DataFrame(SVM_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
TM_concat = pd.concat([TM_SVM], keys=['SVM'])

#Training Accuracies
TA_SVM = pd.Series(SVM_trainingAccuracy)
TA_concat = pd.DataFrame(pd.concat([TA_SVM],ignore_index=True), columns=(['Tr_Accuracy']))\
                .rename({0:'SVM'})

#Validation-Error Matrices
VM_SVM = pd.DataFrame(SVM_errorMatrix).rename(columns=rowIndex, index=rowIndex)
VM_concat = pd.concat([VM_SVM], keys=['SVM'])

#Validation Accuracies
VA_SVM = pd.Series(SVM_errorAccuracy)
VA_concat = pd.DataFrame(pd.concat([VA_SVM],ignore_index=True), columns=(['Va_Accuracy']))\
                .rename({0:'SVM'})

#Producer-User Accuracies
PU_concat = pd.concat([PU_SVM], keys=['SVM'])

#Kappa coefficients
Kp_SVM = pd.Series(SVM_kappa)
Kp_concat = pd.DataFrame(pd.concat([Kp_SVM],ignore_index=True), columns=(['Kappa']))\
                .rename({0:'SVM'})

Kp_concat.style.set_caption('Kappa coefficients')
#print(Kp_concat)

Extract the number of training and validation points per class:

In [ ]:
trainingInfo = trainingData.aggregate_histogram('class').getInfo()
validationInfo = validationData.aggregate_histogram('class').getInfo()

traSeries = pd.Series(trainingInfo)
valSeries = pd.Series(validationInfo)

Points_concat = pd.DataFrame(pd.concat([traSeries, valSeries],ignore_index=True,axis=1))\
                .rename(columns={0:'TraPoints',1:'ValPoints'}).rename({'0':'Sb','1':'Hb','2':'Dn'},axis='index')
print(Points_concat)

Organize each matrix in separate excel sheets

In [ ]:
excelName = 'Mrx'+ smoothStr + imageID +'.xlsx'
excel = pd.ExcelWriter(excelName, engine='xlsxwriter')

Points_concat.to_excel(excel, sheet_name='Points', index=True, startrow=0)
TM_concat.to_excel(excel, sheet_name='TrMrx', index=True, startrow=0)
TA_concat.to_excel(excel, sheet_name='TrAcc', index=True, startrow=0)
VM_concat.to_excel(excel, sheet_name='VaMrx', index=True, startrow=0)
VA_concat.to_excel(excel, sheet_name='VaAcc', index=True, startrow=0)
PU_concat.to_excel(excel, sheet_name='PU-Mrx', index=True, startrow=0)
Kp_concat.to_excel(excel, sheet_name='Kappa', index=True, startrow=0)

Save matrices as .xlsx file:

In [ ]:
excel.save()
print('SAVED')